In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from scipy import stats
from scipy import stats
import math
from sklearn.preprocessing import MinMaxScaler,PolynomialFeatures, StandardScaler
from sklearn import tree,linear_model,neighbors
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline
import warnings 
warnings.filterwarnings("ignore")
sns.set(style="white", color_codes=True)
sns.set_context(rc={"font.family":'sans',"font.size":10,"axes.titlesize":24,"axes.labelsize":24}) 
from sklearn.linear_model import Lasso

# Upload data

In [2]:
train = pd.read_csv('TrainClean.csv',index_col=0).reset_index(drop=True)
test = pd.read_csv('TestClean.csv',index_col=0).reset_index(drop=True)
train.describe()

,IsHoliday,HasPromotions,NearestCompetitor,Region_AreaKM2,Region_GDP,Region_PopulationK,Mean_Dew_PointC,Mean_Humidity,Mean_Sea_Level_PressurehPa,Mean_TemperatureC,...,December,Fog,Nothing,Rain,Thunderstorm,Snow,Hail,ZeroClouds,NumberOfSales,NumberOfCustomers
count,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.00000,...,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000,433958.000000
mean,0.000945,0.449507,8042.993882,12313.341916,14045.556837,3845.765475,6.285320,74.859115,1014.979698,10.33585,...,0.077973,0.254421,0.238873,0.592698,0.061453,0.076784,0.006415,0.000129,2.239679,312.547433
std,0.030723,0.497444,11640.203554,9114.075588,2760.416990,2816.324768,5.862926,12.928407,8.262854,7.04847,...,0.268129,0.435536,0.426396,0.491333,0.240160,0.266249,0.079839,0.011359,0.050325,158.106619
min,0.000000,0.000000,47.000000,344.000000,9893.000000,816.000000,-15.000000,30.000000,974.000000,-13.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1057.000000,7215.000000,11849.000000,1293.000000,2.000000,66.000000,1010.000000,4.00000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.211314,214.000000
50%,0.000000,0.000000,3307.000000,9337.000000,15017.000000,1892.000000,7.000000,76.000000,1015.000000,11.00000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.241269,278.000000
75%,0.000000,1.000000,9761.000000,15566.000000,15931.000000,5727.000000,11.000000,85.000000,1020.000000,16.00000,...,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.270551,369.000000
max,1.000000,1.000000,85070.000000,32221.000000,23931.000000,8146.000000,20.000000,100.000000,1040.000000,30.00000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.415042,2206.000000


In [3]:
train.columns

Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_GDP', 'Region_PopulationK', 'Mean_Dew_PointC', 'Mean_Humidity',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Mean_VisibilityKm',
       'Mean_Wind_SpeedKm_h', 'Precipitationmm', 'StandardMarket',
       'HyperMarket', 'SuperMarket', 'ShoppingCenter', 'General',
       'WithFishDepartment', 'WithNonFoodDepartment', 'Region0', 'Region1',
       'Region2', 'Region3', 'Region4', 'Region5', 'Region6', 'Region7',
       'Region8', 'Region9', 'January', 'February', 'March', 'April', 'May',
       'June', 'July', 'August', 'September', 'October', 'November',
       'December', 'Fog', 'Nothing', 'Rain', 'Thunderstorm', 'Snow', 'Hail',
       'ZeroClouds', 'NumberOfSales', 'NumberOfCustomers'],
      dtype='object')

# Normalization

In [4]:
all_data_index = train.columns.drop(['NumberOfSales','NumberOfCustomers'])
all_data = pd.concat((train.loc[:,all_data_index],test.loc[:,all_data_index]), ignore_index=True)
columns = all_data.columns

In [5]:
x = all_data.values
min_max_scaler = MinMaxScaler()
all_data_normalized = pd.DataFrame(min_max_scaler.fit_transform(x))
all_data_normalized.columns = columns

In [6]:
test_offset = train.shape[0]
train_normalized = all_data_normalized.iloc[0:test_offset,:]
train_normalized['NumberOfSales'] = train["NumberOfSales"]
train_normalized['NumberOfCustomers'] = train["NumberOfCustomers"]

In [7]:
test_normalized = all_data_normalized.iloc[test_offset:all_data_normalized.shape[0],:]
train=train_normalized
test=test_normalized

# Lasso for Number Of Customers

In [19]:
y=train['NumberOfCustomers'].values.reshape(-1,1)
x=train.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)

In [20]:
NumOfCustF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    
    if abs(c)>threshold and not i == 0:
        NumOfCustF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfCustF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.321
	                  Train RMSE=16981.830

Linear Regression with Lasso - Chosen Features
HasPromotions	63.246
NearestCompetitor	-102.751
Region_AreaKM2	-52.519
Mean_Dew_PointC	-13.654
Mean_Sea_Level_PressurehPa	12.577
Mean_VisibilityKm	1.168
Precipitationmm	-21.329
StandardMarket	-66.951
HyperMarket	-11.400
ShoppingCenter	293.108
General	-25.889
WithFishDepartment	373.501
Region0	5.987
Region1	-18.504
Region3	50.425
Region4	88.786
Region5	-19.120
Region6	-28.981
Region7	-24.684
Region8	-57.956
Region9	9.543
January	-9.032
February	-7.175
April	3.059
May	10.251
June	4.644
September	-4.643
October	-2.917
November	0.538
December	50.819
Fog	-1.644
Nothing	1.202
Snow	-5.067


Index(['HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Mean_Dew_PointC', 'Mean_Sea_Level_PressurehPa', 'Mean_VisibilityKm',
       'Precipitationmm', 'StandardMarket', 'HyperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'Region0', 'Region1', 'Region3',
       'Region4', 'Region5', 'Region6', 'Region7', 'Region8', 'Region9',
       'January', 'February', 'April', 'May', 'June', 'September', 'October',
       'November', 'December', 'Fog', 'Nothing', 'Snow'],
      dtype='object')

# Lasso for number of sales with direct passage

In [8]:
y=train['NumberOfSales'].values.reshape(-1,1)
x=train.drop(columns=['NumberOfCustomers','NumberOfSales'],axis=1)

In [9]:
NumOfCustF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    
    if abs(c)>threshold and not i == 0:
        NumOfCustF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfCustF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.229
	                  Train RMSE=0.002

Linear Regression with Lasso - Chosen Features
HasPromotions	0.038
NearestCompetitor	-0.003
Region_GDP	0.006
Mean_Sea_Level_PressurehPa	0.006
Mean_TemperatureC	-0.005
Precipitationmm	-0.005
StandardMarket	-0.000
HyperMarket	-0.004
SuperMarket	0.000
ShoppingCenter	0.032
General	-0.017
WithFishDepartment	0.003
Region0	0.005
Region1	-0.000
Region2	0.005
Region3	0.024
Region4	0.012
Region5	-0.004
Region6	-0.008
Region8	-0.011
Region9	0.009
January	-0.002
February	-0.001
March	-0.001
May	0.003
June	0.002
July	0.000
September	-0.003
October	-0.002
November	0.002
December	0.024
Fog	-0.001
Snow	-0.001


Index(['HasPromotions', 'NearestCompetitor', 'Region_GDP',
       'Mean_Sea_Level_PressurehPa', 'Mean_TemperatureC', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'General', 'WithFishDepartment', 'Region0', 'Region1', 'Region2',
       'Region3', 'Region4', 'Region5', 'Region6', 'Region8', 'Region9',
       'January', 'February', 'March', 'May', 'June', 'July', 'September',
       'October', 'November', 'December', 'Fog', 'Snow'],
      dtype='object')

# Normalization of number of customers

In [21]:
x = train['NumberOfCustomers'].values
min_max_scaler = MinMaxScaler()
train['NumberOfCustomers'] = min_max_scaler.fit_transform(x.reshape(-1,1))
x = train.drop(columns=['NumberOfSales'],axis=1)
y =train['NumberOfSales'].values.reshape(-1,1)

# Lasso for Number Of Sales

In [22]:
NumOfSalesF = []

lasso = linear_model.LassoCV(cv=10)
lasso_fit=lasso.fit(x,y)
yp_lasso = lasso.predict(x)
r2_simple_train = r2_score(y,yp_lasso)
rmse_simple_train = mean_squared_error(y,yp_lasso,multioutput='raw_values')
print("Linear Regression with Lasso")
print("==================================================")
print("\t                  Train R2=%.3f"%(r2_simple_train))
print("\t                  Train RMSE=%.3f"%(rmse_simple_train))
print("\nLinear Regression with Lasso - Chosen Features")
selected=[]
threshold=0
for i,c in enumerate(np.append(lasso_fit.intercept_,lasso_fit.coef_)):
    #print("%d\t%3.3f" % (i,c) )
    if abs(c)>threshold and not i == 0:
        NumOfSalesF.append(x.columns[i-1])
        print("%s\t%3.3f" % (x.columns[i-1],c) )
        
z=x[NumOfSalesF]
z.columns

Linear Regression with Lasso
	                  Train R2=0.665
	                  Train RMSE=0.001

Linear Regression with Lasso - Chosen Features
IsHoliday	-0.008
HasPromotions	0.022
NearestCompetitor	0.021
Region_AreaKM2	0.012
Region_PopulationK	0.008
Mean_Humidity	0.003
Mean_Sea_Level_PressurehPa	0.004
Mean_TemperatureC	-0.007
Mean_Wind_SpeedKm_h	0.000
Precipitationmm	-0.001
StandardMarket	0.017
HyperMarket	-0.000
SuperMarket	0.001
ShoppingCenter	-0.041
WithFishDepartment	-0.073
WithNonFoodDepartment	0.010
Region0	0.003
Region1	0.002
Region2	0.003
Region3	0.011
Region4	-0.004
Region6	-0.003
Region7	0.006
Region8	0.003
January	-0.002
February	-0.002
March	-0.003
April	-0.002
May	0.000
June	0.001
July	0.001
September	-0.003
October	-0.002
November	0.001
December	0.009
Fog	-0.001
Nothing	-0.000
Rain	0.000
Thunderstorm	0.000
Snow	-0.000
NumberOfCustomers	0.558


Index(['IsHoliday', 'HasPromotions', 'NearestCompetitor', 'Region_AreaKM2',
       'Region_PopulationK', 'Mean_Humidity', 'Mean_Sea_Level_PressurehPa',
       'Mean_TemperatureC', 'Mean_Wind_SpeedKm_h', 'Precipitationmm',
       'StandardMarket', 'HyperMarket', 'SuperMarket', 'ShoppingCenter',
       'WithFishDepartment', 'WithNonFoodDepartment', 'Region0', 'Region1',
       'Region2', 'Region3', 'Region4', 'Region6', 'Region7', 'Region8',
       'January', 'February', 'March', 'April', 'May', 'June', 'July',
       'September', 'October', 'November', 'December', 'Fog', 'Nothing',
       'Rain', 'Thunderstorm', 'Snow', 'NumberOfCustomers'],
      dtype='object')

In [23]:
len(NumOfSalesF)

41

In [24]:
len(NumOfCustF)

33